<a href="https://colab.research.google.com/github/prateekrao/Breast-Cancer-Detection/blob/main/resnet-best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip "/content/drive/MyDrive/split-data.zip"

In [1]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, Flatten, GlobalAveragePooling2D
from keras.applications.densenet import DenseNet169, DenseNet121
from keras.applications import ResNet101
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l1, l2, l1_l2
import matplotlib.pyplot as plt

In [3]:
traingen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 45
)

In [4]:
train_set = traingen.flow_from_directory(
    "/content/split-data/train",
    target_size = (50,50),
    color_mode= 'rgb',
    class_mode = 'sparse',
    shuffle = True,
)

Found 194266 images belonging to 2 classes.


In [5]:
valid_set = traingen.flow_from_directory(
    "/content/split-data/valid",
    target_size = (50,50),
    class_mode = 'sparse',
    color_mode = 'rgb',
    shuffle = True
)

Found 41629 images belonging to 2 classes.


In [6]:
testgen = ImageDataGenerator(
    rescale = 1./255
)

In [7]:
test_set = testgen.flow_from_directory(
    "/content/split-data/test",
    target_size = (50,50),
    class_mode = 'sparse',
    color_mode = 'rgb',
    shuffle = False
)

Found 41629 images belonging to 2 classes.


In [8]:
def build_model():

  resnet = ResNet101(include_top=False, weights = 'imagenet', input_shape = (50,50,3))
  for layer in resnet.layers[-50:-1]:
    layer.trainable = False

  model = tf.keras.models.Sequential()

  model.add(resnet)
  model.add(GlobalAveragePooling2D())
  
  model.add(Dense(512, activation='relu', kernel_regularizer=l1_l2(l1=1e-04, l2=1e-04)))
  model.add(Dropout(0.5))
  model.add(Dense(512, activation= 'relu', kernel_regularizer=l1_l2(l1 = 1e-05, l2= 1e-05)))
  #model.add(Dropout(0.5))
  #model.add(Dense(512, activation='relu', kernel_regularizer=l1_l2(l1=1e-03, l2=1e-03)))
  #model.add(Dropout(0.5))
  #model.add(Dense(512, activation='relu', kernel_regularizer=l1_l2(l1 = 1e-02, l2=1e-02)))
  model.add(Dense(2, activation='softmax'))
  
  return model

In [9]:

model = build_model()
model.compile(
      optimizer=tf.keras.optimizers.Adam(lr = 6e-05),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])

171450368/171446536 [==============================] - 1s 0us/step


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Functional)       (None, 2, 2, 2048)        42658176  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 43,970,946
Trainable params: 26,915,074
Non-trainable params: 17,055,872
___________________________________

In [11]:
history = model.fit(train_set, validation_data=valid_set, epochs = 20, shuffle = True, batch_size = 32)

Epoch 1/20
6071/6071 [==============================] - 579s 89ms/step - loss: 1.9691 - sparse_categorical_accuracy: 0.8527 - val_loss: 0.4785 - val_sparse_categorical_accuracy: 0.8851
Epoch 2/20
6071/6071 [==============================] - 521s 86ms/step - loss: 0.4188 - sparse_categorical_accuracy: 0.8872 - val_loss: 0.3329 - val_sparse_categorical_accuracy: 0.8945
Epoch 3/20
6071/6071 [==============================] - 522s 86ms/step - loss: 0.3182 - sparse_categorical_accuracy: 0.8943 - val_loss: 0.3605 - val_sparse_categorical_accuracy: 0.8740
Epoch 4/20
6071/6071 [==============================] - 522s 86ms/step - loss: 0.2828 - sparse_categorical_accuracy: 0.8972 - val_loss: 0.2998 - val_sparse_categorical_accuracy: 0.8875
Epoch 5/20
6071/6071 [==============================] - 522s 86ms/step - loss: 0.2609 - sparse_categorical_accuracy: 0.9014 - val_loss: 0.2628 - val_sparse_categorical_accuracy: 0.8989
Epoch 6/20
6071/6071 [==============================] - 522s 86ms/step - lo

In [12]:
model.evaluate(test_set)

1301/1301 [==============================] - 46s 35ms/step - loss: 0.3087 - sparse_categorical_accuracy: 0.8979


[0.30868151783943176, 0.8979077339172363]